In [1]:
import numpy as np
import random

In [15]:
ambiente = [[[], [], [], []], 
            [[], [], [], []],
            [[], [], [], []],
            [[], [], [], []]]

posicoes = [[l, c] for l in range(3) for c in range(1, 4)]
wumpus = random.sample(posicoes, 1)

ambiente[wumpus[0][0]][wumpus[0][1]].append("wumpus")

if wumpus[0][0] != 0:
    ambiente[wumpus[0][0] - 1][wumpus[0][1]].append("fedor")
if wumpus[0][0] != 3:
    ambiente[wumpus[0][0] + 1][wumpus[0][1]].append("fedor")
if wumpus[0][1] != 0:
    ambiente[wumpus[0][0]][wumpus[0][1] - 1].append("fedor")
if wumpus[0][1] != 3:
    ambiente[wumpus[0][0]][wumpus[0][1] + 1].append("fedor")

posicoes.remove(wumpus[0])

abismo = random.sample(posicoes, 3)

for i in range(0, 3):
    ambiente[abismo[i][0]][abismo[i][1]].append("abismo")

    if abismo[i][0] != 0:
        if "brisa" not in ambiente[abismo[i][0] - 1][abismo[i][1]]:
            ambiente[abismo[i][0] - 1][abismo[i][1]].append("brisa")
    if abismo[i][0] != 3:
        if "brisa" not in ambiente[abismo[i][0] + 1][abismo[i][1]]:
            ambiente[abismo[i][0] + 1][abismo[i][1]].append("brisa")
    if abismo[i][1] != 0:
        if "brisa" not in ambiente[abismo[i][0]][abismo[i][1] - 1]:
            ambiente[abismo[i][0]][abismo[i][1] - 1].append("brisa")
    if abismo[i][1] != 3:
        if "brisa" not in ambiente[abismo[i][0]][abismo[i][1] + 1]:
            ambiente[abismo[i][0]][abismo[i][1] + 1].append("brisa")

for i in abismo:
    posicoes.remove(i)

ouro = random.sample(posicoes, 1)

ambiente[ouro[0][0]][ouro[0][1]].append("ouro")

for i in ambiente:
    print(i)

[[], ['fedor'], ['wumpus', 'brisa'], ['fedor', 'abismo']]
[[], ['brisa', 'ouro'], ['fedor', 'abismo'], ['brisa']]
[[], [], ['brisa'], ['abismo']]
[[], [], [], ['brisa']]


In [3]:
n_states = 16     # número de casas
n_actions = 4     # para cima, para baixo, para esquerda e para direita
learning_rate = 0.1
discount_factor = 0.9
exploration_prob = 0.2
n_episodes = 10     # número de episódios de treinamento

Q = np.zeros((n_states, n_actions))     # inicializando a tabela Q

In [4]:
# os índices da numeração das casas do ambiente são os estados da tabela Q
def index_to_coords(index):
    return divmod(index, 4)
# retorna o quociente e o resto da divisão do índice por 4
# que correspondem às coordenadas x e y do índice, respectivamente

In [5]:
def coords_to_index(i, j):
    return i * 4 + j

In [6]:
# função para escolher uma ação com base na política epsilon greedy
def choose_action(state):
    # gera um número aleatório entre 0 e 1, se ele for menor que exploration_prob (probabilidade de explorar aleatoriamente), é escolhida uma ação aleatória
    if random.uniform(0, 1) < exploration_prob:
        return random.choice(range(n_actions))
    # caso contrário, ele escolhe a ação com maior valor Q naquele estado (está fora da probabilidade de escolha de ação aleatória)
    else:
        return np.argmax(Q[state, :])     # retorna o índice do maior valor

In [7]:
# determina o próximo estado com a partir da ação tomada
def next_state(current_state, action):
    act = action
    i, j = index_to_coords(current_state)
    if act == 0 and i > 0:
        i -= 1     # cima
    elif act == 1 and j < 3:
        j += 1     # direita
    elif act == 2 and i < 3:
        i += 1     # baixo
    elif act == 3 and j > 0:
        j -= 1     # esquerda
    return coords_to_index(i, j)

In [16]:
x, y = (3, 0)
state = coords_to_index(x, y)     # estado inicial

for episode in range(n_episodes):
    while "ouro" not in ambiente[x][y]:     # enquanto não chegar ao destino
        action = choose_action(state)
        next = next_state(state, action)
        x_next, y_next = index_to_coords(next)

        if "ouro" in ambiente[x_next][y_next]:
            reward = 10
        elif "wumpus" in ambiente[x_next][y_next] or "abismo" in ambiente[x_next][y_next]:
            reward = -10
        elif "fedor" in ambiente[x_next][y_next] or "brisa" in ambiente[x_next][y_next] or next == state:
        # com next == state penaliza quando uma ação que não pode ser tomada é escolhida e o agente se mantém no mesmo estado
            reward = -1
        else:
            reward = 0

        Q[state, action] = Q[state, action] + learning_rate * (reward + discount_factor * np.max(Q[next, :]) - Q[state, action])
        state = next
        x, y = index_to_coords(state)

In [17]:
optimal_policy = np.argmax(Q, axis=1)
# retorna a ação (coluna) com maior valor Q para cada estado (linha)
print("Política ótima (ações 0=cima, 1=direita, 2=baixo, 3=esquerda):")
print(optimal_policy.reshape(4, 4))
# reorganiza o vetor de ações (optimal_policy) em uma matriz 4x4, correspondente à grade do ambiente
# assim, na posição de cada casa do ambiente fica o valor correspondente à melhor ação

Política ótima (ações 0=cima, 1=direita, 2=baixo, 3=esquerda):
[[1 2 1 0]
 [1 0 0 0]
 [2 1 0 0]
 [0 0 0 0]]


In [18]:
for i in ambiente:
    print(i)

[[], ['fedor'], ['wumpus', 'brisa'], ['fedor', 'abismo']]
[[], ['brisa', 'ouro'], ['fedor', 'abismo'], ['brisa']]
[[], [], ['brisa'], ['abismo']]
[[], [], [], ['brisa']]
